# Stock Anlysis

Preparing Stock Data

In [2]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from IPython.display import clear_output

# importing list of stock tickers
NASDAQ_tickers_list = pd.read_csv('nasdaq_tickers.csv')

# getting dates to observe (past 30 days)
todays_date = date.today()
from_date = date.today() - timedelta(days=29)

# creating dictionary to store dataframes
stock_data = {}

ticker_index = 1
for stock in NASDAQ_tickers_list['Symbol']:
    print(ticker_index, '/', len(NASDAQ_tickers_list['Symbol']), ' COMPLETED', sep=None)
    try:
        data = yf.download(stock, from_date, todays_date, progress=False)
        if not data.empty:
            stock_data[stock] = {
                'Company Name': yf.Ticker(stock).info['longName'],
                'Stock Data': data,
                'Sharpe Ratio': -256,
                'Treynor Ratio': -256
            }
            print(f'{stock} DOWNLOADED!')
    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")
    clear_output(wait=True)
    ticker_index += 1

print('Stock Download COMPLETE')

Stock Download COMPLETE


Calculating Sharpe Ratio

In [7]:
# Getting risk-free rate
treasury_ticker = "^TNX"
treasury_data = yf.download(treasury_ticker, todays_date, progress=False)
risk_free_rate = treasury_data['Close'].iloc[-1] / 100
risk_free_rate_daily = risk_free_rate / 252

# Calculating each the Sharpe Ratio for each stock
for stock in stock_data:
    stock_data[stock]['Stock Data']['Daily Return'] = stock_data[stock]['Stock Data']['Adj Close'].pct_change()
    average_return = stock_data[stock]['Stock Data']['Daily Return'].mean()

    std_dev_return = 0
    if len(stock_data[stock]['Stock Data']['Daily Return'].array) > 1:  # Ensure there are enough data points
        std_dev_return = stock_data[stock]['Stock Data']['Daily Return'].std()
    else:
        # print(f'ERROR - {stock}: Not enough information to calculate risk!')
        continue

    if std_dev_return == 0:
        # print(f'{stock}: Consistent Close of 0 -- Skipping Sharpe Ratio calculation!')
        continue

    stock_data[stock]['Sharpe Ratio'] = (average_return - risk_free_rate_daily) / std_dev_return

# Sorting and printing 10 highest Sharpe Ratio stocks
stock_sorted_sharpe_ratio = dict(sorted(stock_data.items(), key=lambda item: item[1]['Sharpe Ratio'], reverse=True))
for i, stock in enumerate(stock_sorted_sharpe_ratio):
    if i > 10:
        break
    print(stock, ', ', stock_sorted_sharpe_ratio[stock]['Company Name'], ': ', stock_sorted_sharpe_ratio[stock]['Sharpe Ratio'], sep='')

CNFRZ, Conifer Holdings, Inc.: 1.2172605715959994
RPHM, Reneo Pharmaceuticals, Inc.: 0.9110002991219545
CRMD, CorMedix Inc.: 0.8314883270081891
PHUN, Phunware, Inc.: 0.6620343175346237
NN, NextNav Inc.: 0.635982911466526
MYRG, MYR Group Inc.: 0.5979175442587367
LLYVA, Liberty Live Group: 0.5929089620659632
LLYVK, Liberty Live Group: 0.589055580496083
NCNO, nCino, Inc.: 0.5687561696138758
PAYO, Payoneer Global Inc.: 0.5552994618334922
LOGC, ContextLogic Inc.: 0.5532681497015639


Calculating Treynor Ratio

In [8]:
import numpy as np

# Fetching market data
market_data = yf.download('^GSPC', start=from_date, end=todays_date, progress=False)
market_data['Daily Return'] = market_data['Adj Close'].pct_change().dropna()

for stock in stock_data:
    # Calculating beta
    if len(stock_data[stock]['Stock Data']['Daily Return'].dropna()) != len(market_data['Daily Return'].dropna()):
        continue

    covariance = np.cov(stock_data[stock]['Stock Data']['Daily Return'].dropna(), market_data['Daily Return'].dropna())[0][1]
    market_variance = np.var(market_data['Daily Return'])
    beta = covariance / market_variance

    if beta == 0:
        continue
    
    # Calculating Treynor Ratio
    stock_data[stock]['Treynor Ratio'] = (average_return - risk_free_rate) / beta

stock_sorted_treynor_ratio = dict(sorted(stock_data.items(), key=lambda item: item[1]['Treynor Ratio'], reverse=True))
for i, stock in enumerate(stock_sorted_treynor_ratio):
    if i > 10:
        break
    print(stock, ', ', stock_sorted_treynor_ratio[stock]['Company Name'], ': ', stock_sorted_treynor_ratio[stock]['Treynor Ratio'], sep='')

CDTX, Cidara Therapeutics, Inc.: 1118.724298768857
MYPS, PLAYSTUDIOS, Inc.: 18.644138844617682
MDBH, MDB Capital Holdings, LLC: 11.248592554499691
MTRX, Matrix Service Company: 9.458274607217998
RFACU, RF Acquisition Corp.: 9.248116537310938
GHIX, Gores Holdings IX, Inc.: 9.072573265642315
PLAO, Patria Latin American Opportunity Acquisition Corp.: 7.686418901531902
MOBX, Mobix Labs, Inc.: 7.25048681877758
IVCP, Swiftmerge Acquisition Corp.: 6.989467875655463
SWKHL, SWK Holdings Corporation 9.00% Senior Notes due 2027: 6.174865594882665
FTII, FutureTech II Acquisition Corp.: 6.030620040890418


# Gathering Sentiment Data

NewsAPI

In [1]:
# limited to 100 calls every 24 hours (will take 40 days to get all news articles

from newsapi import NewsApiClient
NEWSAPI_KEY = '81a66bfe85d7441bab96bbda4d1f1e0d'
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

articles_by_name = newsapi.get_everything(
    q=stock_data['APPL']['Company Name'],
    to=todays_date.isoformat(),
    from_param=from_date.isoformat(),
    language="en",
    sort_by="relevancy"
)

for article in articles_by_name['articles']:
    print(article['title']+ ' | ' + article['publishedAt'] + ' | ' + article['url'])

# for stock in stock_data:
#     articles_by_name = newsapi.get_everything(
#         q=stock_data[stock]['Company Name'],
#         to = todays_date.isoformat(),
#         from_param= from_date.isoformat(),
#         language="en",
#         sort_by="relevancy"
#     )

#     print(f'{stock}')
#     for article in articles_by_name['articles']:
#         print(article['title']+ ' | ' + article['publishedAt'] + ' | ' + article['url'])
#     print('------------------------------------------------------------------')


/Users/ethanwright/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'stock_data' is not defined

Twitter Web Scraping

In [ ]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import csv

api = API()
